In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")


*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [2]:
import os

from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json
import fbnsdkutilities.utilities as utils


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = utils.ApiClientFactory(
    lu,
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.11275.0,0.5.3237,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
# here's some sample equities we've prepared earlier
uk_equity_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
us_equity_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [4]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

scope = 'FBNUniversityModule-T2-2' # str | The scope of the portfolio.

In [5]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi},
                                     "Isin":{"value":isin},
                                     other_identifier_name:{"value":other_identifier}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [6]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're going to load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in uk_equity_instruments.itertuples())
uk_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in us_equity_instruments.itertuples())
us_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = us_equity_body_dict | uk_equity_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [7]:

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open('_data/instrument_subset.csv').read()

drive_config = lusid_drive.Configuration(
    host = f'{api_instance.api_client.configuration.host[:-4]}/drive'
)
drive_config.access_token = api_factory.api_client.configuration.access_token

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        
        
    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = 'instrument_subset.csv' # str | File name.
    x_lusid_drive_path = '/finbourne university/' # str | File path.
    content_length = len(subset_file.encode('UTF-8'))
    body = subset_file # str | 

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        

# Luminesce Syntax

In this part of the course we will provide some additional technical information about luminesce, and demonstrate how luminesce queries can be used to extract information from your data.

In [8]:
# Enable luminesce magic
import os
from IPython.core.magic import (register_line_cell_magic)
from lumipy.client import Client
from lusidjam import RefreshingToken

token = RefreshingToken()
lusid_api_url = api_factory.api_client.configuration.host
lumi_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "honeycomb"
os.environ["FBN_LUMI_API_URL"] = lumi_api_url
lumi_url = os.getenv("FBN_LUMI_API_URL")

@register_line_cell_magic
def luminesce(line, cell=None):
    query = cell if cell is not None else line

    lm_client = Client(token=token, api_url=lumi_url)

    df = lm_client.query_and_fetch(query)
            
    return df

# In an interactive session, we need to delete to avoid
# name conflicts for automagic to work on line magics.
del luminesce

## What is Structured Query Language (SQL)


SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language. Many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.

## Luminesce Queries - SQL with extensions


SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.



## Luminesce providers


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself, for files stored in Drive and for some external data providers. You can also create your own provider for a proprietary data source.

There are two types of provider: a data provider and a direct provider.

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

We'll use both data and direct providers in this tutorial.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.


I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

### A query using a simple SELECT statement


Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [9]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Isin,Sedol,Cusip,Ticker,...,CountryOfIncorporation,Exchange,IssuerId,IssuerName,LegalEntityId,NaicsCode,ParValue,PrimaryCurrency,PrimaryExchange,SecurityId
0,LUID_00017YYD,NaN,Equity2c52df3c-7093-41a0-b0c5-e59231c63e23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_000189RT,NaN,Equityf63343b9-e124-4c4d-a6d3-a7eaab067688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_00019T66,NaN,luminesce-test-create-equity-16470116882,NaN,NaN,NaN,US34532RZZ99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LUID_00017PWN,NaN,Equity_LN_GB00BKKMKR23,NaN,NaN,NaN,GB00BKKMKR23,BKKMKR2,NaN,RSA,...,NaN,NaN,NaN,RSA Insurance Group,NaN,NaN,NaN,NaN,NaN,NaN
4,LUID_00017PXJ,NaN,Equity_LN_GB00B03MLX29,NaN,NaN,NaN,GB00B03MLX29,B03MLX2,NaN,RDSA,...,NaN,NaN,NaN,Royal Dutch Shell,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_000189RS,NaN,Equityc3e2bea3-a47b-4489-9fd0-802c134270dd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,LUID_00019YPF,NaN,luminesce-test-create-equity-16470274623,NaN,NaN,NaN,US34532RZZ99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,LUID_00018R5F,NaN,luminesce-test-create-equity-16467561673,NaN,NaN,NaN,US34532RZZ99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,LUID_00016QON,NaN,Equity8bd550c6-96d1-466c-9939-4166dc3dea2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In this example, we've selected all columns and rows from the Lusid.Instrument.Equity provider. We can specify column names if we are only interested in some parts of the data.

In [10]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity
LIMIT 100

,DisplayName,Figi
0,luminesce-test-create-equity-16468383413,NaN
1,luminesce-test-create-equity-16463702571,NaN
2,AstraZeneca1988,BBG00WGHTKZ1988
3,luminesce-test-create-equity-16466577941,NaN
4,AstraZeneca220,BBG00WGHTKZ220
...,...,...
95,AstraZeneca2272,BBG00WGHTKZ2272
96,National Grid plc,NaN
97,AstraZeneca1307,BBG00WGHTKZ1307
98,luminesce-test-create-equity-16460712841,NaN


Here I really just want to see the name of the Equity and the Figi, I've selected only those columns from the provider.

### Limiting the number of return results using LIMIT


These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [11]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity 
LIMIT 5

,DisplayName,Figi
0,Taylor Wimpey,BBG000BF4KL1
1,Equity,NaN
2,Equity,NaN
3,Equity,NaN
4,Equity,NaN


This example returns the first 5 rows of data from the Lusid.Instrument.Equity provider.

### Filter queries WHERE ...


We can now filter these results down - lets get all of the Equities that are listed in GBP


In [12]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity 
WHERE DomCcy = 'GBP'
LIMIT 100

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Isin,Sedol,Cusip,Ticker,...,CountryOfIncorporation,Exchange,IssuerId,IssuerName,LegalEntityId,NaicsCode,ParValue,PrimaryCurrency,PrimaryExchange,SecurityId
0,LUID_HOYPKW9W,BBG000BF2FG0,NaN,NaN,NaN,NaN,GB00BWFGQN14,BWFGQN1,NaN,SPX LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_00015YRE,NaN,luminesce-test-create-equity-16458071312,NaN,NaN,NaN,US34532RZZ99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_00015V7F,NaN,luminesce-test-create-equity-16457932833,NaN,NaN,NaN,US34532RZZ99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LUID_PXFI2CHY,BBG000BD42C6,NaN,NaN,NaN,NaN,GB0000536739,53673,NaN,AHT LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LUID_KDLAKU1A,BBG000BDCN13,NaN,NaN,NaN,NaN,IE0001827041,182704,NaN,CRH LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_R1D3075T,BBG000BZZ876,NaN,NaN,NaN,NaN,GB00B1YW4409,B1YW440,NaN,III LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,LUID_QFJSCWZG,BBG000C04GS5,NaN,NaN,NaN,NaN,ES0177542018,B5M6XQ7,NaN,IAG LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,LUID_00015W87,NaN,luminesce-test-create-equity-16457992193,NaN,NaN,NaN,US34532RZZ99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,LUID_0000MIOW,BBG000D03XD4,imd_43532542,NaN,NaN,NaN,GB00B2B0DG97,NaN,NaN,REL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We've filtered our query for equities who have a domestic currency of Pound Sterling.

### JOINing disparate datasources with join


Now, I've got a file in drive with sector information for some of these instruments. I'm going to join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which SQLite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [13]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector 
FROM Lusid.Instrument.Equity as Equities 
CROSS JOIN @x as File
LIMIT 100

,DisplayName,DomCcy,Sector
0,Taylor Wimpey,GBP,Financial Services
1,Taylor Wimpey,GBP,General Industrials
2,Taylor Wimpey,GBP,Travel & Leisure
3,Taylor Wimpey,GBP,Equity Investment Instruments
4,Taylor Wimpey,GBP,Household Goods & Home Construction
...,...,...,...
95,Equity,USD,Mining
96,Equity,USD,Media
97,Equity,USD,Mobile Telecommunications
98,Equity,USD,Food & Drug Retailers


We've returned a table with every combination of Lusid Equities with every row in our file.

In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

In [14]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM @x as File
LEFT OUTER JOIN Lusid.Instrument.Equity as Equities
on Equities.Figi = File.Figi
LIMIT 100

,DisplayName,DomCcy,Sector,Figi
0,Schroders,GBP,Financial Services,BBG000BF0TF3
1,Schroders,GBP,Financial Services,BBG000BF0TF3
2,Schroders,GBP,Financial Services,BBG000BF0TF3
3,DS Smith,GBP,General Industrials,BBG000BF1LF9
4,DS Smith,GBP,General Industrials,BBG000BF1LF9
...,...,...,...,...
83,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
84,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
85,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7
86,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7


We see this here as there are some companies with no matching sector in our file - these are returned as null.

We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [15]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM @x as File 
INNER JOIN Lusid.Instrument.Equity as Equities  on File.Figi = Equities.Figi 
LIMIT 100


,DisplayName,DomCcy,Sector,Figi
0,Schroders,GBP,Financial Services,BBG000BF0TF3
1,Schroders,GBP,Financial Services,BBG000BF0TF3
2,Schroders,GBP,Financial Services,BBG000BF0TF3
3,DS Smith,GBP,General Industrials,BBG000BF1LF9
4,DS Smith,GBP,General Industrials,BBG000BF1LF9
...,...,...,...,...
83,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
84,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
85,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7
86,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7


Now we have a joined table where only Equities with a sector on file are returned, along with their sector.

### GROUPing results BY some condition in query


We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

There are many different aggregation functions in SQLite, such as sum, avg and count.

Sum will return the sum of the specified field in each group.
Average returns the mean of the specified field in each group.
Count returns the number of results that are non-null for the field in each group. We can also specify that count returns the count of distinct values [Count(Distinct <field>)], or all rows [count(*)], in each group.

In [16]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM @x as File
LEFT OUTER JOIN Lusid.Instrument.Equity as Equities
on Equities.Figi = File.Figi
group by File.Sector
LIMIT 100

,count(File_Sector COLLATE BINARY),Sector
0,2,Automobiles & Parts
1,3,Equity Investment Instruments
2,5,Financial Services
3,3,Food & Drug Retailers
4,3,General Industrials
5,4,Household Goods & Home Construction
6,3,Investment Services
7,13,Life Insurance
8,3,Media
9,7,Mining


We've now grouped our results by sector, and for each sector, we''ve returned how many Equities belong to the sector.

## Introducing some Luminesce extensions


### Using ^ to SELECT the most important columns from a provider


Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [17]:
%%luminesce
SELECT ^ 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,State
0,LUID_6PJJMTTS,BBG000BF4KL1,INTERNAL_ID_5,NaN,NaN,NaN,default,Taylor Wimpey,Equity,Equities,GBP,True,Active
1,LUID_000123Y0,NaN,Equity41027787-3cf6-4dc2-a9d3-a4986f911d02,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active
2,LUID_00011HQT,NaN,Equity8bdba031-d52f-4b7c-8c24-5b7e85e18307,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive
3,LUID_0000YZK3,NaN,Equity8bf6f1ca-a254-4087-b0dc-aa8e9fcce401,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive
4,LUID_000140VI,NaN,Equityd374d886-224f-43bd-abf0-580f1c05fcc6,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_0000XLQX,NaN,Equity43721138-b60c-468b-bf36-4ea3e3bcc9f4,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active
96,LUID_0000WWIL,NaN,Equity037f16cd-ce22-4cec-b5f7-13c8e47a07a6,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive
97,LUID_00015JSI,NaN,Equityf5ffffaf-c6c1-4dd5-a13a-2a645c00144c,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive
98,LUID_000119GL,NaN,Equity6ccc9918-f5bf-4532-9423-d58f76d6e751,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active


In the Lusid Equities provider: LusidInstrumentId, Scope, DisplayName, InferredAssetClass, InferredDomCcy and IsActive are the fields marked by the provider as being most important.

### Excluding columns from the results of a query using the except keyword.


Using some providers, we can also exclude columns from the result of a query by using the except keyword:

In [18]:
%%luminesce
SELECT ^ (except Type, IsActive) 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Scope,DisplayName,InferredAssetClass,InferredDomCcy,State
0,LUID_HOYPKW9W,BBG000BF2FG0,NaN,NaN,NaN,NaN,default,Spirax-Sarco Engineering,Equities,GBP,Active
1,LUID_0000XSVD,NaN,Equity5d312fb0-ee2c-49ca-9b01-a4dfb173f468,NaN,NaN,NaN,default,Equity,Equities,USD,Inactive
2,LUID_0000ZTIA,NaN,Equity3adcb97f-4aa3-477a-9f24-401e25988f13,NaN,NaN,NaN,default,Equity,Equities,USD,Active
3,LUID_000117II,NaN,Equity59963159-1a9c-4666-bc29-5eac75a8a804,NaN,NaN,NaN,default,Equity,Equities,USD,Inactive
4,LUID_00015YRE,NaN,luminesce-test-create-equity-16458071312,NaN,NaN,NaN,default,luminesce-test-create-equity-16458071312,Equities,GBP,Active
...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_000105BJ,NaN,Equity8a14725e-42e8-49d7-9527-d439b013819d,NaN,NaN,NaN,default,Equity,Equities,USD,Active
96,LUID_00013ZIS,NaN,Equity699f9cc2-5194-4319-a4f1-f5154cd895ae,NaN,NaN,NaN,default,Equity,Equities,USD,Inactive
97,LUID_0000Y8SC,NaN,Equity3b799b49-319b-45b0-af88-d3ebadd3a2b7,NaN,NaN,NaN,default,Equity,Equities,USD,Active
98,LUID_0000YIOU,NaN,Equityabcdcdc3-e5fa-4531-a671-04c676bcc9bd,NaN,NaN,NaN,default,Equity,Equities,USD,Active


Here we select the most important columns again, but exclude the Type and IsActive columns

### Using parameters to filter queries


Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the Sys.Field Provider.

In [19]:
%%luminesce
SELECT FieldName, DataType, ParamDefaultValue, Description 
FROM Sys.Field 
WHERE TableName = 'Lusid.Instrument.Equity' 
AND FieldType = 'Parameter'
LIMIT 100

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


We can see there are 3 fields we can filter our query on: AsAt, EffectiveAt and UseLusidFilter

In [20]:
%%luminesce

SELECT * 
FROM Lusid.Instrument.Equity
WHERE UseLusidFilter = False
LIMIT 100

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Isin,Sedol,Cusip,Ticker,...,CountryOfIncorporation,Exchange,IssuerId,IssuerName,LegalEntityId,NaicsCode,ParValue,PrimaryCurrency,PrimaryExchange,SecurityId
0,LUID_6PJJMTTS,BBG000BF4KL1,INTERNAL_ID_5,NaN,NaN,NaN,GB0008782301,878230,NaN,TW/ LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_000123Y0,NaN,Equity41027787-3cf6-4dc2-a9d3-a4986f911d02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_00011HQT,NaN,Equity8bdba031-d52f-4b7c-8c24-5b7e85e18307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LUID_0000YZK3,NaN,Equity8bf6f1ca-a254-4087-b0dc-aa8e9fcce401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LUID_000140VI,NaN,Equityd374d886-224f-43bd-abf0-580f1c05fcc6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_0000XLQX,NaN,Equity43721138-b60c-468b-bf36-4ea3e3bcc9f4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,LUID_0000WWIL,NaN,Equity037f16cd-ce22-4cec-b5f7-13c8e47a07a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,LUID_00015JSI,NaN,Equityf5ffffaf-c6c1-4dd5-a13a-2a645c00144c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,LUID_000119GL,NaN,Equity6ccc9918-f5bf-4532-9423-d58f76d6e751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here we've filtered an Equity query on the UseLusidFilter. Notice there is no returned Field called UseLusidFilter - its not a column in our data.

## Using Luminesce variables


You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. 

In [21]:
%%luminesce
@instruments_in_GBP = SELECT * 
                      FROM Lusid.Instrument.Equity 
                      WHERE DomCcy = 'GBP'
                      LIMIT 100;
SELECT Isin 
FROM @instruments_in_GBP;

,Isin
0,US34532RZZ99
1,GB00BWFGQN14
2,US34532RZZ99
3,US34532RZZ99
4,GB00B39J2M42
...,...
90,GB00BYT1DJ19
91,GB00B0LCW083
92,GB0006731235
93,US34532RZZ99


Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [22]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * 
FROM @sectors 
WHERE Sector = @@selected_sector
LIMIT 100

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


I've set selected_sector as a single-value variable "Lise Insurance", and filtered my file by sector.

## Introducing more advanced Luminesce extensions


You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.

### Cross-apply

Cross-apply is similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
We'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [23]:
%%luminesce

select VALUE as val
from Tools.Split 
where Original = 'GBP USD CHF' 
AND Delimiters = ' ';


,val
0,GBP
1,USD
2,CHF


First, we use Tools.Split to turn a space separated string of currencies into into a table where each currency is an entry in the table.

In [24]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';

SELECT c.val as selectedCurrency, results.*
FROM @currencies c
CROSS APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity equity
       WHERE equity.DomCcy = c.val
       LIMIT 100       
) results


,selectedCurrency,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,State,__row_key__
0,GBP,LUID_6PJJMTTS,BBG000BF4KL1,INTERNAL_ID_5,NaN,NaN,NaN,default,Taylor Wimpey,Equity,Equities,GBP,True,Active,1
1,GBP,LUID_ZCJA2AFZ,BBG000FV67Q4,INTERNAL_ID_3,NaN,NaN,NaN,default,National Grid plc,Equity,Equities,GBP,True,Active,1
2,GBP,LUID_00015YRD,NaN,luminesce-test-create-equity-16458071311,NaN,NaN,NaN,default,luminesce-test-create-equity-16458071311,Equity,Equities,GBP,True,Active,1
3,GBP,LUID_00015VCN,NaN,luminesce-test-create-equity-16457982062,NaN,NaN,NaN,default,luminesce-test-create-equity-16457982062,Equity,Equities,GBP,True,Active,1
4,GBP,LUID_VBRC6JMF,BBG00RM7J0M5,NaN,NaN,NaN,NaN,default,JUST EAT TAKEAWAY,Equity,Equities,GBP,True,Active,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,USD,LUID_000105BJ,NaN,Equity8a14725e-42e8-49d7-9527-d439b013819d,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active,2
98,USD,LUID_00013ZIS,NaN,Equity699f9cc2-5194-4319-a4f1-f5154cd895ae,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive,2
99,USD,LUID_0000Y8SC,NaN,Equity3b799b49-319b-45b0-af88-d3ebadd3a2b7,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active,2
100,USD,LUID_0000YIOU,NaN,Equityabcdcdc3-e5fa-4531-a671-04c676bcc9bd,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active,2


We then save the result of that query in a variable @currency, which is passed as a criterion in the CROSS APPLY subquery,  which runs in parallel for each row in @currency.

This example works similar to an inner join, we've run a query in parallel for some currency values, returning a similar table to an inner join. Remember, inner join returns only rows which fulfil the join criteria.

### Outer-apply


Outer apply is similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.

In [25]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';
              
SELECT c.val, results.*
FROM @currencies c
OUTER APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity as equity
       WHERE equity.DomCcy = c.val
       LIMIT 100
) results


,val,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,State,__row_key__
0,GBP,LUID_00015UBH,NaN,luminesce-test-create-equity-16457919263,NaN,NaN,NaN,default,luminesce-test-create-equity-16457919263,Equity,Equities,GBP,True,Active,1.0
1,GBP,LUID_00015V7F,NaN,luminesce-test-create-equity-16457932833,NaN,NaN,NaN,default,luminesce-test-create-equity-16457932833,Equity,Equities,GBP,True,Active,1.0
2,GBP,LUID_00015VCQ,NaN,luminesce-test-create-equity-16457982372,NaN,NaN,NaN,default,luminesce-test-create-equity-16457982372,Equity,Equities,GBP,True,Active,1.0
3,GBP,LUID_00015W86,NaN,luminesce-test-create-equity-16457992191,NaN,NaN,NaN,default,luminesce-test-create-equity-16457992191,Equity,Equities,GBP,True,Active,1.0
4,GBP,LUID_00015YRC,NaN,luminesce-test-create-equity-16458071313,NaN,NaN,NaN,default,luminesce-test-create-equity-16458071313,Equity,Equities,GBP,True,Active,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,USD,LUID_000164HJ,NaN,Equity87cf9b7e-6bef-47de-86f2-d6d8e457c572,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active,2.0
95,USD,LUID_000164LG,NaN,Equity5fc18630-e1b5-4d04-ba55-4577e30f7b2c,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive,2.0
96,USD,LUID_000165ZM,NaN,Equitye575356a-478e-4b7c-b097-08b2a7480c58,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,False,Inactive,2.0
97,USD,LUID_000165ZN,NaN,Equity55bab978-f260-4209-9861-e6ea52cf0ab3,NaN,NaN,NaN,default,Equity,Equity,Equities,USD,True,Active,2.0


We've run a query in parallel for some currency values, returning a similar table to a left join with our selected currencies on the left side of the join. Here, the values from Lusid.Instrument.Equity on the CHF row are null.